In [1]:
import pandas as pd

In [2]:
import os
os.getcwd()

'c:\\Users\\jjjen\\추천시스템\\Audit'

In [3]:
df = pd.read_csv('news_preprocessed_0620.csv')
df.head()

,keyword,title,date,cleaned_content
0,"""한국가스공사"" + 손실","[단독] ‘30조 적자’ 한전, 빚내 민간발전사 손실 메운다",2023-05-10 04:09:12,SMP 상한제로 1조대 손실 추산구입 155원 판매 120원 역마진구조금주 전기가...
1,"""한국가스공사"" + 손실",한전·가스공사 손실 쌓이는데...2분기 요금 두고 국회·세종·용산은 혼돈 속,2023-05-11 19:01:01,관련법규정에 없는 당정협의로 2분기 요금 40일째 미궁20조 자구책에도 여당 뼈 ...
2,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 06:01:03,12일께 1분기 실적 공개올해 8조7천억원 적자 전망가스공사 미수금도 11조원대로...
3,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 08:54:43,원가보다 싼 전력 공급의 장기화로 적자가 불어나고 있는 한국전력은 올해 1...
4,"""한국가스공사"" + 손실",與 “文정부 탓 한전 추가손실…요금 올리려면 한전 자구노력 선행돼야”,2023-04-25 06:02:02,에너지미래포럼 25일 에너지정책 토론회박대출 국민의힘 정책위 의장 재차 강조에너지...


In [4]:
df = df.dropna()

# Kobert

In [5]:
import torch
import tensorflow as tf
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm.auto import tqdm

In [6]:
print(f'Current device: {torch.cuda.get_device_name(torch.cuda.current_device())}')

if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")


Current device: NVIDIA GeForce RTX 3060 Laptop GPU
GPU is available


# KOBERT

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [27]:
# Check if a GPU is available and if not, use a CPU

device = torch.device("cuda")
        
# Load the pre-trained model
model = BertForSequenceClassification.from_pretrained("monologg/kobert").to(device)

# Load the pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")

# Create a new `output` column and initialize it to 0
df["output_kobert"] = 0.0

# Add a progress bar
pbar = tqdm(total=len(df))

for i, row in df.iterrows():
    # Input text
    text = row["cleaned_content"]

    # Tokenize the text with the tokenizer and convert it to the model's input format
    input_ids = tokenizer.encode(text, add_special_tokens=True)

    # Truncate the input_ids to 512 tokens if it's longer
    #if len(input_ids) > 512:
    #    input_ids = input_ids[:512]

    # Add padding or truncate to max length
    input_ids = input_ids[:512] if len(input_ids) > 512 else input_ids + [0] * (512 - len(input_ids))

    input_ids = torch.tensor([input_ids]).to(device)

    # Predict with the model
    outputs = model(input_ids)

    # Convert the result from log probabilities to probabilities
    probs = torch.nn.functional.softmax(outputs[0], dim=1)

    # Store the negative probability in the `output` column
    df.at[i, "output_kobert"] = probs[0][1].item()

    # Update the progress bar
    pbar.update()

# Close the progress bar
pbar.close()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/104290 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors


# KLUE

In [28]:
device = torch.device("cuda")

# Load the pre-trained model and move it to the device
model = BertForSequenceClassification.from_pretrained("klue/bert-base").to(device) 

# Load the pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

# Create a new `output` column and initialize it to 0
df["output_klue"] = 0.0

# Add a progress bar
pbar = tqdm(total=len(df))

for i, row in df.iterrows():
    # Input text
    text = row["cleaned_content"]

    # Tokenize the text with the tokenizer and convert it to the model's input format
    input_ids = tokenizer.encode(text, add_special_tokens=True)

    # Truncate the input_ids to 512 tokens if it's longer
    input_ids = input_ids[:512] if len(input_ids) > 512 else input_ids + [0] * (512 - len(input_ids))

    input_ids = torch.tensor([input_ids]).to(device)

    # Predict with the model
    outputs = model(input_ids)

    # Convert the result from log probabilities to probabilities
    probs = torch.nn.functional.softmax(outputs[0], dim=1)

    # Store the negative probability in the `output` column
    df.at[i, "output_klue"] = probs[0][1].item()

    # Update the progress bar
    pbar.update()

# Close the progress bar
pbar.close()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initial

  0%|          | 0/104290 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (612 > 512). Running this sequence through the model will result in indexing errors


In [11]:
df.head()

,keyword,title,date,cleaned_content,output,output_kobert
0,"""한국가스공사"" + 손실","[단독] ‘30조 적자’ 한전, 빚내 민간발전사 손실 메운다",2023-05-10 04:09:12,SMP 상한제로 1조대 손실 추산구입 155원 판매 120원 역마진구조금주 전기가...,0.558287,0.459686
1,"""한국가스공사"" + 손실",한전·가스공사 손실 쌓이는데...2분기 요금 두고 국회·세종·용산은 혼돈 속,2023-05-11 19:01:01,관련법규정에 없는 당정협의로 2분기 요금 40일째 미궁20조 자구책에도 여당 뼈 ...,0.463708,0.473964
2,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 06:01:03,12일께 1분기 실적 공개올해 8조7천억원 적자 전망가스공사 미수금도 11조원대로...,0.556163,0.476445
3,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 08:54:43,원가보다 싼 전력 공급의 장기화로 적자가 불어나고 있는 한국전력은 올해 1...,0.450437,0.437062
4,"""한국가스공사"" + 손실",與 “文정부 탓 한전 추가손실…요금 올리려면 한전 자구노력 선행돼야”,2023-04-25 06:02:02,에너지미래포럼 25일 에너지정책 토론회박대출 국민의힘 정책위 의장 재차 강조에너지...,0.535732,0.473413


# 추가 처리

In [12]:
# 문자열 분할
df['기업'], df['kwd'] = zip(*df['keyword'].apply(lambda x: x.split(' + ', 1)))

# 따옴표 제거
df['기업'] = df['기업'].str.strip('"')

df.head()

,keyword,title,date,cleaned_content,output,output_kobert,기업,kwd
0,"""한국가스공사"" + 손실","[단독] ‘30조 적자’ 한전, 빚내 민간발전사 손실 메운다",2023-05-10 04:09:12,SMP 상한제로 1조대 손실 추산구입 155원 판매 120원 역마진구조금주 전기가...,0.558287,0.459686,한국가스공사,손실
1,"""한국가스공사"" + 손실",한전·가스공사 손실 쌓이는데...2분기 요금 두고 국회·세종·용산은 혼돈 속,2023-05-11 19:01:01,관련법규정에 없는 당정협의로 2분기 요금 40일째 미궁20조 자구책에도 여당 뼈 ...,0.463708,0.473964,한국가스공사,손실
2,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 06:01:03,12일께 1분기 실적 공개올해 8조7천억원 적자 전망가스공사 미수금도 11조원대로...,0.556163,0.476445,한국가스공사,손실
3,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 08:54:43,원가보다 싼 전력 공급의 장기화로 적자가 불어나고 있는 한국전력은 올해 1...,0.450437,0.437062,한국가스공사,손실
4,"""한국가스공사"" + 손실",與 “文정부 탓 한전 추가손실…요금 올리려면 한전 자구노력 선행돼야”,2023-04-25 06:02:02,에너지미래포럼 25일 에너지정책 토론회박대출 국민의힘 정책위 의장 재차 강조에너지...,0.535732,0.473413,한국가스공사,손실


In [13]:
df.기업.unique()

array(['한국가스공사', '케이티앤지', '하나투어', '신세계인터내셔날', '카카오', 'LG유플러스', '현대백화점',
       '한섬', '한국투자금융지주', '강원랜드', '네이버', '현대로템', '한국항공우주산업', '키움증권',
       '한화시스템', '롯데관광개발', '에이치디현대인프라코어', '대우조선해양', '대우건설', '포스코인터내셔널',
       'LG화학', 'LG생활건강', '현대글로비스', '현대홈쇼핑', '신한금융지주회사', 'LG전자', '티케이지휴켐스',
       '대웅제약', '금호타이어', '엘아이지넥스원', 'GS', '셀트리온', '농심', '삼양식품', 'CJ',
       '오리온홀딩스', '롯데지주', '대상', '오뚜기', '두산', '롯데칠성음료', '하이트진로', '효성',
       '태광산업', '지누스', '대한유화', '금양', 'LG', '한화솔루션', '롯데케미칼', '롯데정밀화학',
       'DB하이텍', '한올바이오파마', '녹십자홀딩스', '대웅', '보령', '유한양행', '한미사이언스',
       '아모레퍼시픽그룹', 'SKC', '한화', 'S-OIL', '한국앤컴퍼니', '한솔케미칼', '금호석유화학',
       '신풍제약', '녹십자', '아이에스동서', '쌍용씨앤이', '동서', '동국제강', '현대제철', '영풍',
       '코스모화학', '케이지스틸', '고려아연', '포스코홀딩스', '롯데에너지머티리얼즈', '한샘', '현대모비스',
       '삼성중공업', '현대엘리베이터', 'LS', '대한전선', '삼성SDI', '하나금융지주', '엘지이노텍',
       '삼성전자', '삼성전기', '한화에어로스페이스', '현대미포조선', '기아', '현대자동차', '현대위아',
       '세방전지', '에스엘', '두산에너빌리티', '에이치디한국조선해양', '에스케이하이닉스', '엘지디스플레이',
       '명신산업', '하이브'

In [14]:
df = df.rename(columns = {'output':'output_klue'})
df.head()

,keyword,title,date,cleaned_content,output_klue,output_kobert,기업,kwd
0,"""한국가스공사"" + 손실","[단독] ‘30조 적자’ 한전, 빚내 민간발전사 손실 메운다",2023-05-10 04:09:12,SMP 상한제로 1조대 손실 추산구입 155원 판매 120원 역마진구조금주 전기가...,0.558287,0.459686,한국가스공사,손실
1,"""한국가스공사"" + 손실",한전·가스공사 손실 쌓이는데...2분기 요금 두고 국회·세종·용산은 혼돈 속,2023-05-11 19:01:01,관련법규정에 없는 당정협의로 2분기 요금 40일째 미궁20조 자구책에도 여당 뼈 ...,0.463708,0.473964,한국가스공사,손실
2,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 06:01:03,12일께 1분기 실적 공개올해 8조7천억원 적자 전망가스공사 미수금도 11조원대로...,0.556163,0.476445,한국가스공사,손실
3,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 08:54:43,원가보다 싼 전력 공급의 장기화로 적자가 불어나고 있는 한국전력은 올해 1...,0.450437,0.437062,한국가스공사,손실
4,"""한국가스공사"" + 손실",與 “文정부 탓 한전 추가손실…요금 올리려면 한전 자구노력 선행돼야”,2023-04-25 06:02:02,에너지미래포럼 25일 에너지정책 토론회박대출 국민의힘 정책위 의장 재차 강조에너지...,0.535732,0.473413,한국가스공사,손실


In [15]:
len(df)

104290

In [16]:
df = df.dropna()
df.isnull().sum()

keyword            0
title              0
date               0
cleaned_content    0
output_klue        0
output_kobert      0
기업                 0
kwd                0
dtype: int64

In [17]:
df = df.query('date != "None"')
df.head()

,keyword,title,date,cleaned_content,output_klue,output_kobert,기업,kwd
0,"""한국가스공사"" + 손실","[단독] ‘30조 적자’ 한전, 빚내 민간발전사 손실 메운다",2023-05-10 04:09:12,SMP 상한제로 1조대 손실 추산구입 155원 판매 120원 역마진구조금주 전기가...,0.558287,0.459686,한국가스공사,손실
1,"""한국가스공사"" + 손실",한전·가스공사 손실 쌓이는데...2분기 요금 두고 국회·세종·용산은 혼돈 속,2023-05-11 19:01:01,관련법규정에 없는 당정협의로 2분기 요금 40일째 미궁20조 자구책에도 여당 뼈 ...,0.463708,0.473964,한국가스공사,손실
2,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 06:01:03,12일께 1분기 실적 공개올해 8조7천억원 적자 전망가스공사 미수금도 11조원대로...,0.556163,0.476445,한국가스공사,손실
3,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 08:54:43,원가보다 싼 전력 공급의 장기화로 적자가 불어나고 있는 한국전력은 올해 1...,0.450437,0.437062,한국가스공사,손실
4,"""한국가스공사"" + 손실",與 “文정부 탓 한전 추가손실…요금 올리려면 한전 자구노력 선행돼야”,2023-04-25 06:02:02,에너지미래포럼 25일 에너지정책 토론회박대출 국민의힘 정책위 의장 재차 강조에너지...,0.535732,0.473413,한국가스공사,손실


In [18]:
df['date_new'] = pd.to_datetime(df['date'], errors='coerce').dt.date # 날짜와 시간으로 변환
df.head()


,keyword,title,date,cleaned_content,output_klue,output_kobert,기업,kwd,date_new
0,"""한국가스공사"" + 손실","[단독] ‘30조 적자’ 한전, 빚내 민간발전사 손실 메운다",2023-05-10 04:09:12,SMP 상한제로 1조대 손실 추산구입 155원 판매 120원 역마진구조금주 전기가...,0.558287,0.459686,한국가스공사,손실,2023-05-10
1,"""한국가스공사"" + 손실",한전·가스공사 손실 쌓이는데...2분기 요금 두고 국회·세종·용산은 혼돈 속,2023-05-11 19:01:01,관련법규정에 없는 당정협의로 2분기 요금 40일째 미궁20조 자구책에도 여당 뼈 ...,0.463708,0.473964,한국가스공사,손실,2023-05-11
2,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 06:01:03,12일께 1분기 실적 공개올해 8조7천억원 적자 전망가스공사 미수금도 11조원대로...,0.556163,0.476445,한국가스공사,손실,2023-05-09
3,"""한국가스공사"" + 손실","'전기 팔수록 손해' 한전, 올 1분기 5조원대 영업손실 예고",2023-05-09 08:54:43,원가보다 싼 전력 공급의 장기화로 적자가 불어나고 있는 한국전력은 올해 1...,0.450437,0.437062,한국가스공사,손실,2023-05-09
4,"""한국가스공사"" + 손실",與 “文정부 탓 한전 추가손실…요금 올리려면 한전 자구노력 선행돼야”,2023-04-25 06:02:02,에너지미래포럼 25일 에너지정책 토론회박대출 국민의힘 정책위 의장 재차 강조에너지...,0.535732,0.473413,한국가스공사,손실,2023-04-25


In [29]:
df.to_csv('output_0620_v2.csv', index = False)